# 1. Tauc-Lorenz model simulation
**Achtung!** To run this notebook your environment needs to have working copies of the hyperspy and relativistic_eels modules.

This notebook creates an EELS-SL with spectra generated by the same complex dielectric function (CDF), with a linear gradient of thickness. The CDF is generated from an empiric model. We use the Tauc-Lorenz model, that contains plasmon resonance and band gap transition terms. Hence, this model is specially well suited to test relativistic KKA as we can control the band gap energy and the correction of relativistic losses at the level of band gap is our objective.

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt

from model import *
from time import time

# Generate dielectric response models

de = 0.05

# Inputs: array size dependent
#thk = np.linspace(20, 50, 21)
thk = np.linspace(20, 120, 21)
gap = np.array([1., 3., 5.])
eax = np.arange(de, 80., de)

# Inputs: dielectric properties
resonance = 6.
damping   = 1.
n_bandgap = 3.5
n_plasmon = 7.

# Inputs spectrum simulation
Izlp = 1e6   # zero-loss intensity, 1e4=>SNR=20dB, noisy
nang = 256   # number of angles
tmin = 1e-4  # theta-min for integration in mrad
nois = True  # add poisson noise

# KK params
Izlp       = 1e6        # zero-loss intensity, 1e4=>SNR=20dB, noisy
nang       = 256        # number of angles
tmin       = 1e-3       # theta-min for integration
nois       = True       # add poisson noise
chi2target = 5e-4
# optionally smooth
fsmooth    = 0.2

# dielectric properties
cdf, t = dielectric_tauc_bandgap(eax,
                                 gap,
                                 thk,
                                 resonance,
                                 damping,
                                 n_plasmon,
                                 n_bandgap)

t = cdf._check_adapt_map_input(
    cdf.axes_manager[0].axis * np.ones_like(cdf.axes_manager[1].axis)[:,None] )

# DDCS
tempo = time()
Ptot, Pvol, Pelf = cdf.get_relativistic_ddcs(t, theta_min=tmin, NA=nang)
tempo = time() - tempo
print('DDCS calculation time', tempo)

# Spectrum simulation
tempo = time()
Itot, Ivol, Ielf =  cdf.get_relativistic_spectrum(Izlp, t, ddcs=[Ptot, Pvol, Pelf])
tempo = time() - tempo
print('Spectrum integration time', tempo)

Ptot.data = Ptot.data.astype('float32')
Pvol.data = Pvol.data.astype('float32')
Pelf.data = Pelf.data.astype('float32')

Itot.data = Itot.data.astype('float32')
Ivol.data = Ivol.data.astype('float32')
Ielf.data = Ielf.data.astype('float32')


# Simulate Poisson noise
Iexp = Itot.deepcopy()
if nois:
    Iexp.add_poissonian_noise()
Iexp.remove_negative_intensity(True)
Iexp.data += 1e-5

In [ ]:
cdf.plot()

In [ ]:
from signals import Signal1DComparator

scomp = Signal1DComparator([Itot, Ielf, Iexp], 
                           legend=[r'$S(E)$', r'$S_b^{ELF}(E)$', r'$S(E)$, noisy'])

In [ ]:
scomp = Signal1DComparator([Itot, Ivol, Itot-Ivol], 
                           legend=[r'$S(E)$', r'$S_b(E)$', r'$S_s(E)$'])

## Save the results:

In [ ]:
sfile = './example_data/'
cdf.save(sfile+'cdf')
Ptot.save(sfile+'Ptot')
Pvol.save(sfile+'Pvol')
Pelf.save(sfile+'Pelf')
Itot.save(sfile+'Itot')
Ivol.save(sfile+'Ivol')
Ielf.save(sfile+'Ielf')
Iexp.save(sfile+'Iexp')

The notebook may now be restarted to save memory

# 2. Relativistic Kramers-Kronig analysis
Using the simulated files from the previous run, try reconstructing the CDF using relativistic Kramers-Kronig algorithm. **Be careful!** some of the parameters need to be equal to the parameters chosen here (this is indicated below).

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
from model import *
from time import time

# simulation params (need to be the same as above)
Izlp = 1e6   # zero-loss intensity, 1e4=>SNR=20dB, noisy
nang = 256   # number of angles
tmin = 1e-4  # theta-min for integration in mrad
nois = True  # add poisson noise
sfile='./example_data/'

# KK params
Izlp       = 1e6        # zero-loss intensity, 1e4=>SNR=20dB, noisy
nang       = 256        # number of angles
tmin       = 1e-3       # theta-min for integration
nois       = True       # add poisson noise
chi2target = 5e-4

# load the data
cdf = ModifiedCDF(hs.load(sfile+'cdf.hspy'))
Iexp = ModifiedEELS(hs.load(sfile+'Iexp.hspy'))

gaps = Iexp.axes_manager[1].axis
thk = Iexp.axes_manager[0].axis
t = Iexp._check_adapt_map_input( thk * np.ones_like(gaps)[:,None] )

# classic KKA
eps_kka, out = Iexp.kramers_kronig_analysis(Izlp, 5, None, t, full_output=True)
Icor_kka = out['surface plasmon estimation']
eps_kka.data = eps_kka.data.astype('complex64')

# for rKKA in general
t_i = t.inav[:, 0]

# select naive rKKA
use_average = False
fsmooth     = None

eps_rkka = cdf.deepcopy()
Icor_rkka = Iexp.deepcopy()
for io in range(len(gaps)):
    Iexp_i = Iexp.inav[:, io]
    eps_i, out = Iexp_i.relativistic_kramers_kronig(Izlp,
                                                None,
                                                t_i,
                                                fsmooth=fsmooth,
                                                theta_min=tmin,
                                                NA=nang,
                                                average=use_average,
                                                chi2_target=chi2target)
    corr_iter = out['relativistic correction']

    eps_rkka.data[io, :, :] = eps_i.data.astype('complex64')
    Icor_rkka.data[io, :, :] = corr_iter.data.astype('float32')

# select regularized rKKA
use_average = False
fsmooth     = 0.2

eps_rkka_fs = cdf.deepcopy()
Icor_rkka_fs = Iexp.deepcopy()
for io in range(len(gaps)):
    Iexp_i = Iexp.inav[:, io]
    eps_i, out = Iexp_i.relativistic_kramers_kronig(Izlp,
                                                None,
                                                t_i,
                                                fsmooth=fsmooth,
                                                theta_min=tmin,
                                                NA=nang,
                                                average=use_average,
                                                chi2_target=chi2target)
    corr_iter = out['relativistic correction']

    eps_rkka_fs.data[io, :, :] = eps_i.data.astype('complex64')
    Icor_rkka_fs.data[io, :, :] = corr_iter.data.astype('float32')

# select average rKKA
use_average = True
fsmooth     = 0.2

eps_rkka_fs_avg = cdf.deepcopy()
Icor_rkka_fs_avg = Iexp.deepcopy()
for io in range(len(gaps)):
    Iexp_i = Iexp.inav[:, io]
    eps_i, out = Iexp_i.relativistic_kramers_kronig(Izlp,
                                                None,
                                                t_i,
                                                fsmooth=fsmooth,
                                                theta_min=tmin,
                                                NA=nang,
                                                average=use_average,
                                                chi2_target=chi2target)
    corr_iter = out['relativistic correction']

    eps_rkka_fs_avg.data[io, :, :] = eps_i.data.astype('complex64')
    Icor_rkka_fs_avg.data[io, :, :] = corr_iter.data.astype('float32')

In [ ]:
from signals import Signal1DComparator

scomp = Signal1DComparator([cdf.imag, eps_kka.imag, eps_rkka_fs.imag], 
                           legend=[r'$\epsilon_{TL}$','KKA', 'rKKA'])

## save the data

In [ ]:
sfile = './example_data/'
eps_kka.save(sfile+'eps_kka')
Icor_kka.save(sfile+'Icor_kka')
eps_rkka.save(sfile+'eps_rkka')
Icor_rkka.save(sfile+'Icor_rkka')
eps_rkka_fs.save(sfile+'eps_rkka_fs')
Icor_rkka_fs.save(sfile+'Icor_rkka_fs')
eps_rkka_fs_avg.save(sfile+'eps_rkka_fs_avg')
Icor_rkka_fs_avg.save(sfile+'Icor_rkka_fs_avg')

# Compare corrections

In [ ]:
sfile = './example_data/'
Itot = hs.load(sfile+'Itot.hspy')
Ielf = hs.load(sfile+'Ielf.hspy')
Icor_kka = hs.load(sfile+'Icor_kka.hspy')
Icor_rkka_fs = hs.load(sfile+'Icor_rkka_fs.hspy')

In [ ]:
scomp = Signal1DComparator([Itot-Ielf, Icor_kka, Icor_rkka_fs], 
                           legend=[r'$\epsilon_{TL}$','KKA', 'rKKA'])